In [2]:
import pandas as pd
import numpy as np

In [3]:
combined_df = pd.read_csv('./combined_cleaned.csv')

In [5]:
mapping_df = pd.read_csv('./CategoryMapping.csv')

In [6]:
import uuid

# Create a distinct list of categories from the NewCat column, excluding 'Drop'
categories_df = mapping_df[mapping_df['NewCat'] != 'Drop']['NewCat'].drop_duplicates().reset_index(drop=True)

# Create a DataFrame with CategoryId and CategoryName
cat_df = pd.DataFrame({
    'Id': [str(uuid.uuid4()) for _ in range(len(categories_df))],
    'Name': categories_df
})

# Create a mapping dictionary for category names to their GUIDs
cat_mapping = dict(zip(cat_df['Name'], cat_df['Id']))

# Create a distinct list of subcategories from the NewSubCat column, excluding rows where NewCat is 'Drop'
subcategories_df = mapping_df[mapping_df['NewCat'] != 'Drop'][['NewCat', 'NewSubCat']].drop_duplicates().reset_index(drop=True)

# Create a DataFrame with SubCategoryId, SubCategoryName and ParentCategoryId
subcat_df = pd.DataFrame({
    'Id': [str(uuid.uuid4()) for _ in range(len(subcategories_df))],
    'Name': subcategories_df['NewSubCat'],
    'ParentCategoryId': subcategories_df['NewCat'].map(cat_mapping)
})

# Save to CSV files
cat_df.to_csv('./categories.csv', index=False)
cat_df.to_csv('../../src/nimblist/Nimblist.data/Data/categories.csv', index=False)
subcat_df.to_csv('./subcategories.csv', index=False)
subcat_df.to_csv('../../src/nimblist/Nimblist.data/Data/subcategories.csv', index=False)

print(f"Categories created: {len(cat_df)}")
print(f"Subcategories created: {len(subcat_df)}")

# Display samples of both dataframes
print("\nCategories sample:")
print(cat_df.head())

print("\nSubcategories sample:")
print(subcat_df.head())

Categories created: 12
Subcategories created: 145

Categories sample:
                                     Id                  Name
0  27dbcc1a-5d2b-4a40-9e24-50fbfce59aae       Fresh & Chilled
1  a2f0ca2b-9b15-458c-8cf2-310f69eb5899       Treats & Snacks
2  4a4df996-2d61-424d-9af1-7fdfef76e63e                Bakery
3  96607918-0fb6-406e-8198-292c5febc1fb         Food Cupboard
4  85ca5c67-e914-4edf-b7b4-02bd515ad729  Beer, Wine & Spirits

Subcategories sample:
                                     Id                   Name  \
0  9d7a8c89-f46d-4e55-8fd4-0b3d81e23415       Yogurts (Snacks)   
1  a352fe8c-cd12-4b79-a934-6ecf574dbbc5       Yogurts (Snacks)   
2  00c17cf2-1db4-494a-bb15-e5d1b1d42df0     Wraps & Flatbreads   
3  d760320f-113a-4c30-bd99-9b190ed70b65  World Foods (Chilled)   
4  2f0495b3-3a8f-490e-9577-5233612efea6  World Foods (Chilled)   

                       ParentCategoryId  
0  27dbcc1a-5d2b-4a40-9e24-50fbfce59aae  
1  a2f0ca2b-9b15-458c-8cf2-310f69eb5899  
2  4a4df996-

In [9]:
# Create a mapping dictionary for category and subcategory mapping
category_mapping = {}
for _, row in mapping_df.iterrows():
    key = (row['source'], row['parent_category'], row['subcategory'])
    value = (row['NewCat'], row['NewSubCat'])
    category_mapping[key] = value

# Apply the mapping to the combined_df
for index, row in combined_df.iterrows():
    key = (row['source'], row['parent_category'], row['subcategory'])
    if key in category_mapping:
        combined_df.at[index, 'newCat'] = category_mapping[key][0]
        combined_df.at[index, 'newSubCat'] = category_mapping[key][1]

# Display the first few rows to verify
combined_df.head()

# Count products by new categories
print(f"Number of products mapped to categories: {combined_df['newCat'].replace('', 0).replace(0, np.nan).count()}")
print(f"Number of products mapped to 'Drop': {(combined_df['newCat'] == 'Drop').sum()}")

Number of products mapped to categories: 58121
Number of products mapped to 'Drop': 10288


In [10]:
from collections import Counter
import re

# 1. First, let's find the most commonly used words in the cleaned_product_name column

# Extract all words from cleaned_product_name and count them
all_words = ' '.join(combined_df['cleaned_product_name']).lower()
# Remove punctuation and numbers
all_words = re.sub(r'[^\w\s]', ' ', all_words)
all_words = re.sub(r'\d+', ' ', all_words)
# Split into words
words_list = all_words.split()
# Count occurrences
word_counts = Counter(words_list)

# Display the 30 most common words
print("Most common words in product names:")
print(word_counts.most_common(30))



Most common words in product names:
[('g', 15256), ('tesco', 7794), ('ml', 7177), ('s', 4294), ('waitrose', 3774), ('pack', 3560), ('x', 3397), ('free', 2619), ('f', 2552), ('chocolate', 2518), ('organic', 2380), ('chicken', 2068), ('finest', 1755), ('essential', 1697), ('in', 1605), ('no', 1593), ('milk', 1543), ('l', 1501), ('with', 1483), ('cheese', 1456), ('cream', 1430), ('white', 1390), ('original', 1357), ('home', 1338), ('fruit', 1073), ('drink', 1031), ('sauce', 996), ('duchy', 980), ('the', 977), ('black', 967)]


In [11]:
# 2. Create a function to remove specified words from product names
def clean_brand_names(product_name, brands_to_remove):
    """
    Remove brand names and common words from product names
    """
    # Convert to lowercase for case-insensitive matching
    product_name_lower = product_name.lower()
    
    # Create pattern with word boundaries to match whole words only
    pattern = r'\b(' + '|'.join(brands_to_remove) + r')\b'
    
    # Replace matches with empty string
    cleaned = re.sub(pattern, '', product_name_lower, flags=re.IGNORECASE)
    
    # Clean up extra spaces
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()
    
    # Capitalize first letter
    if cleaned:
        cleaned = cleaned[0].upper() + cleaned[1:] if len(cleaned) > 1 else cleaned.upper()
    
    return cleaned

# List of brand names and common words to remove
brands_to_remove = ['waitrose', 'tesco', 'essential', 'finest']

# Apply the function to create a new column
combined_df['generic_product_name'] = combined_df['cleaned_product_name'].apply(
    lambda x: clean_brand_names(x, brands_to_remove)
)

# Display sample results
print("\nSample results after brand name removal:")
sample_df = combined_df[['cleaned_product_name', 'generic_product_name']].head(10)
print(sample_df)

# Count how many products were affected
changed_count = (combined_df['cleaned_product_name'] != combined_df['generic_product_name']).sum()
print(f"\nNumber of product names modified: {changed_count} out of {len(combined_df)} ({changed_count/len(combined_df)*100:.1f}%)")


Sample results after brand name removal:
              cleaned_product_name     generic_product_name
0        Waitrose Cantaloupe Melon         Cantaloupe melon
1     Waitrose Perfectly Ripe Kiwi      Perfectly ripe kiwi
2   Waitrose Seedless Sable Grapes    Seedless sable grapes
3      Essential Fairtrade Bananas        Fairtrade bananas
4      Essential Fairtrade Bananas        Fairtrade bananas
5            Waitrose Loose Lemons             Loose lemons
6          Perfectly Ripe Avocados  Perfectly ripe avocados
7   Waitrose Seedless Easy Peelers    Seedless easy peelers
8  Waitrose Perfectly Ripe Avocado   Perfectly ripe avocado
9     Waitrose Red Seedless Grapes      Red seedless grapes

Number of product names modified: 58006 out of 58121 (99.8%)


In [12]:
# Filter out rows where newCat is "Drop"
combined_df = combined_df[combined_df['newCat'] != "Drop"]

# Reset the index after filtering
combined_df = combined_df.reset_index(drop=True)

# Display the number of rows after filtering
print(f"Number of rows after removing 'Drop' category: {len(combined_df)}")

Number of rows after removing 'Drop' category: 47833


In [13]:
# Remove duplicates from combined_df based on generic_product_name, newCat, and newSubCat
# This helps to eliminate products that are essentially the same but from different sources

# First, let's check how many duplicates we have based on generic_product_name
duplicate_count = combined_df.duplicated(subset=['generic_product_name']).sum()
print(f"Number of duplicate generic product names: {duplicate_count}")

# Create a new column that combines product name and category for better deduplication
combined_df['product_category_key'] = combined_df['generic_product_name'] + ' | ' + combined_df['newCat'] + ' | ' + combined_df['newSubCat']

# Now deduplicate based on this combined key
combined_df_unique = combined_df.drop_duplicates(subset=['product_category_key'], keep='first')

# Remove the temporary column as it's no longer needed
combined_df_unique.drop('product_category_key', axis=1, inplace=True)

# Display information about the deduplication
print(f"Original number of products: {len(combined_df)}")
print(f"Number of unique products after deduplication: {len(combined_df_unique)}")
print(f"Number of duplicates removed: {len(combined_df) - len(combined_df_unique)}")

# Display distribution by source after deduplication
source_counts = combined_df_unique['source'].value_counts()
print("\nSource distribution after deduplication:")
print(source_counts)

# Update the combined_df with the deduplicated version
combined_df = combined_df_unique

# Display sample rows from the deduplicated dataframe
combined_df.head()

Number of duplicate generic product names: 9170
Original number of products: 47833
Number of unique products after deduplication: 45285
Number of duplicates removed: 2548

Source distribution after deduplication:
source
Tesco       28503
Waitrose    16782
Name: count, dtype: int64


C:\Users\ThomasNorton\AppData\Local\Temp\ipykernel_54840\11806896.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df_unique.drop('product_category_key', axis=1, inplace=True)


,product_id,product_name,parent_category,subcategory,source,cleaned_product_name,newCat,newSubCat,generic_product_name
0,568810,Waitrose Cantaloupe Melon,Fresh & Chilled,Fresh Fruit,Waitrose,Waitrose Cantaloupe Melon,Fresh & Chilled,Fresh Fruit,Cantaloupe melon
1,38528,Waitrose Perfectly Ripe Kiwi,Fresh & Chilled,Fresh Fruit,Waitrose,Waitrose Perfectly Ripe Kiwi,Fresh & Chilled,Fresh Fruit,Perfectly ripe kiwi
2,55061,Waitrose Seedless Sable Grapes,Fresh & Chilled,Fresh Fruit,Waitrose,Waitrose Seedless Sable Grapes,Fresh & Chilled,Fresh Fruit,Seedless sable grapes
3,88903,Essential Fairtrade Bananas,Fresh & Chilled,Fresh Fruit,Waitrose,Essential Fairtrade Bananas,Fresh & Chilled,Fresh Fruit,Fairtrade bananas
5,88411,Waitrose Loose Lemons,Fresh & Chilled,Fresh Fruit,Waitrose,Waitrose Loose Lemons,Fresh & Chilled,Fresh Fruit,Loose lemons


In [15]:
combined_df.to_csv('./combined_cleaned.csv', index=False)
combined_df.to_csv('../ClassificationModel/combined_cleaned.csv', index=False)